# Assignment 4: Face Detection and Recognition

## DTSC-680: Applied Machine Learning

## Name: Angelica LOZANO GOMEZ

The `data` directory contains three folders 30 face images of three players from the 2024 Philadelphia Phillies: Alec Bohm, Bryson Stott, and Brandon Marsh. The original images were downloaded from Google Images and the face images were extraced using the `extract_faces` function from the textbook. In addition, the `Samples` directory contains three images of a combination of those players.

Your task is to create a system that can correctly detect and identify Bohm, Stott, and Marsh in those three sample images.

### Install Tensorflow 2.9.2

In [4]:
# CODE PROVIDED
import subprocess
import sys
from IPython.display import clear_output
def check_tensorflow():
    tf_desired_version = "2.9.2"

    try:
        import tensorflow as tf
        tf_installed_version = tf.__version__
    except ImportError:
        tf_installed_version = None

    #Check for the right version
    if tf_installed_version != tf_desired_version:
        print(f"Current TensorFlow version: {tf_installed_version}. Installing version {tf_desired_version}...")
        
        # Uninstall the current TensorFlow version
        subprocess.check_call([sys.executable, '-m', 'pip', 'uninstall', '-y', 'tensorflow'])
        
        # Install the desired TensorFlow version
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', f'tensorflow=={tf_desired_version}'])

        clear_output()
        
        print(f"TensorFlow version {tf_desired_version} installed successfully. Please restart your kernel to apply the changes.")
    else:
        print(f"TensorFlow version {tf_desired_version} is already installed.")    

check_tensorflow()

TensorFlow version 2.9.2 is already installed.


In [5]:
# STUDENT CODE 1
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input as vgg_preprocess

In [6]:
# STUDENT CODE 2
label_map = {
    'Bohm': 0,
    'Stott': 1,
    'Marsh': 2
}
folders_to_use = list(label_map.keys())
train_path = 'data'

total_images = 0
for folder in folders_to_use:
    folder_path = os.path.join(train_path, folder)
    count = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"{folder}: {count} images")
    total_images += count

print("Total training images:", total_images)

Bohm: 10 images
Stott: 10 images
Marsh: 10 images
Total training images: 30


In [7]:
# STUDENT CODE 3
for folder in folders_to_use:
    print(f"Label for {folder}: {label_map[folder]}")

Label for Bohm: 0
Label for Stott: 1
Label for Marsh: 2


In [8]:
# STUDENT CODE 4
from tensorflow.keras.utils import img_to_array, load_img
from keras_vggface.utils import preprocess_input as vgg_preprocess

X = []
y = []

for folder in folders_to_use:
    label = label_map[folder]
    folder_path = os.path.join(train_path, folder)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            full_path = os.path.join(folder_path, filename)
            img = load_img(full_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = vgg_preprocess(np.expand_dims(img_array, axis=0))
            X.append(img_array[0])
            y.append(label)

X = np.array(X)
y = np.array(y)


In [9]:
# STUDENT CODE 5
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
# STUDENT CODE 6
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace

base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True


x = base_model.output
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M2


2025-04-24 08:25:33.765122: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-24 08:25:33.765221: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# STUDENT CODE 7
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define helper used by make_pred
def get_face(np_image, face, required_size=(224, 224)):
    x1, y1, width, height = face['box']
    x2, y2 = x1 + width, y1 + height
    face_crop = np_image[y1:y2, x1:x2]
    image_resized = cv2.resize(face_crop, required_size)
    return image_resized

augmenter = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    augmenter.flow(X_train, y_train, batch_size=4),
    validation_data=(X_test, y_test),
    epochs=100,  # Increased from 50 to 100
    callbacks=[early_stop],
    verbose=2
)

model.save("model.h5")

Epoch 1/100


2025-04-24 08:25:34.683510: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-04-24 08:25:35.715426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2025-04-24 08:25:37.279067: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 - 3s - loss: 4.8418 - accuracy: 0.4583 - val_loss: 2.8473e-04 - val_accuracy: 1.0000 - 3s/epoch - 526ms/step
Epoch 2/100
6/6 - 0s - loss: 1.0750 - accuracy: 0.7083 - val_loss: 2.2668e-05 - val_accuracy: 1.0000 - 281ms/epoch - 47ms/step
Epoch 3/100
6/6 - 0s - loss: 2.4747 - accuracy: 0.7917 - val_loss: 1.1054 - val_accuracy: 0.6667 - 254ms/epoch - 42ms/step
Epoch 4/100
6/6 - 0s - loss: 0.2233 - accuracy: 0.9167 - val_loss: 0.0542 - val_accuracy: 1.0000 - 252ms/epoch - 42ms/step
Epoch 5/100
6/6 - 0s - loss: 0.0862 - accuracy: 0.9583 - val_loss: 3.9736e-08 - val_accuracy: 1.0000 - 255ms/epoch - 42ms/step
Epoch 6/100
6/6 - 0s - loss: 0.3836 - accuracy: 0.9167 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 252ms/epoch - 42ms/step
Epoch 7/100
6/6 - 0s - loss: 0.6660 - accuracy: 0.9167 - val_loss: 1.1921e-07 - val_accuracy: 1.0000 - 246ms/epoch - 41ms/step
Epoch 8/100
6/6 - 0s - loss: 0.6361 - accuracy: 0.7917 - val_loss: 0.4681 - val_accuracy: 0.8333 - 245ms/epoch - 41ms/step
Epoch 9/10

## Codegrade Validation

### Execute the cells below to generate the file required for submission to Codegrade. DO NOT CHANGE THE CODE.

In [13]:
# CODE PROVIDED
import os
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from PIL import Image, ImageOps
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications.resnet50 import preprocess_input


def make_pred(path, model, names, face_threshold=0.9, prediction_threshold=0.9, show_outline=True, size=(12, 8)):
    # Load the image and orient it correctly
    pil_image = Image.open(path)
    exif = pil_image.getexif()
    
    for k in exif.keys():
        if k != 0x0112:
            exif[k] = None
            del exif[k]
            
    pil_image.info["exif"] = exif.tobytes()
    pil_image = ImageOps.exif_transpose(pil_image)
    np_image = np.array(pil_image)

    detector = MTCNN()
    faces = detector.detect_faces(np_image)
    faces = [face for face in faces if face['confidence'] > face_threshold]

    for face in faces:
        x, y, w, h = face['box']

        # Use the model to identify the face
        face_image = get_face(np_image, face)
        face_image = image.array_to_img(face_image)
        face_image = preprocess_input(np.array(face_image))
        predictions = model.predict(np.expand_dims(face_image, axis=0))
        confidence = np.max(predictions)

        if (confidence > prediction_threshold):
            return predictions
            

        return None
        
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

def get_all_images_from_directory(directory):
    images = []
    labels = []
    images_names = []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            images_names += [img for img in os.listdir(label_path)]
            images += [os.path.join(label_path, img) for img in os.listdir(label_path)]
            labels += [int(label)] * len(os.listdir(label_path))
    
    # Embaralhar as imagens e labels
    combined = list(zip(images_names, images, labels))
    random.shuffle(combined)
    images_names, images, labels = zip(*combined)
    
    #return images_names, images, labels
    return zip(*combined)

def predict_and_validate(model, image_names, image_paths, true_labels, class_labels):
    predictions = []
    for img_path in image_paths:
        predictions.append(make_pred(img_path, model, class_labels, prediction_threshold=0.8))
    
    results = [true == pred for true, pred in zip(true_labels, predictions)]
    
    df = pd.DataFrame({
        'Player': image_names,
        'Prediction Array': predictions
    })
    
    return df

In [14]:
# CODE PROVIDED
from IPython.display import clear_output

model_to_evaluate = model  #MAKE SURE TO USE YOUR MODEL

class_labels = {0: 'Alec Bohm', 1: 'Bryson Stott', 2: 'Brandon Marsh'} 
image_names, image_paths, true_labels = get_all_images_from_directory('data/codegrade_test/')
prediction_df = predict_and_validate(model_to_evaluate, image_names, image_paths, true_labels, class_labels)
clear_output()
prediction_df

,Player,Prediction Array
0,Bryson_09.png,"[[2.367299e-11, 1.0, 1.1893617e-21]]"
1,alec_07.jpg,"[[1.0, 1.0928834e-11, 4.5577733e-15]]"
2,alec_02.jpg,"[[1.0, 5.20231e-12, 2.5790213e-16]]"
3,alec_06.jpg,"[[1.0, 2.2493726e-12, 2.7165634e-13]]"
4,Brandon_08.png,"[[1.8212669e-06, 5.541312e-07, 0.9999976]]"
5,alec_01.jpg,"[[1.0, 8.1304775e-14, 2.623304e-13]]"
6,Bryson_04.png,"[[1.4542277e-12, 1.0, 1.5317565e-21]]"
7,alec_05.jpg,"[[0.9999845, 1.5488878e-05, 9.418053e-10]]"
8,Brandon_04.png,"[[4.6691318e-07, 2.124318e-05, 0.9999783]]"
9,Bryson_06.png,"[[4.1001262e-16, 1.0, 3.18551e-20]]"


### Export Prediction Array for Codegrade evaluation


In [16]:
# CODE PROVIDED
import pandas as pd

#export your classification model
prediction_df.to_pickle('prediction_df')